In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re


In [10]:
# Cargar un archivo de datos en formato pickle y guardarlo en el DataFrame `tweets_df`
tweets_df = pd.read_pickle('../data/data_tweets.pkl')

# Establecer la columna 'id_2' como índice del DataFrame
tweets_df.set_index('id_2', inplace=True)

# Eliminar las columnas 'ID_1' y 'type' del DataFrame
tweets_df.drop(['ID_1', 'type'], axis=1, inplace=True)

# Renombrar la columna 'Emprea' a 'Empresa' para corregir el nombre
tweets_df.rename(columns={'Emprea': 'Empresa'}, inplace=True)

# Convertir la columna 'tweetText' al tipo de datos `string`
tweets_df['tweetText'] = tweets_df['tweetText'].astype('string')

tweets_df.head(2)

,Empresa,tweetText,tweetURL,tweetAuthor,handle,replyCount,quoteCount,retweetCount,likeCount,views,bookmarkCount,createdAt,allMediaURL,videoURL,Idioma_1,Idioma_2,Año,Unnamed: 20,Unnamed: 21,openai_tag
id_2,,,,,,,,,,,,,,,,,,,,
948282772120064000,1,"This year, we are looking forward to having mo...",https://twitter.com/abinbev/status/94828277212...,AB InBev,@abinbev,0,0,12,29,-,0,2018-01-02 21:00:05,https://pbs.twimg.com/media/DSj6JxHXcAAu4Jd.jpg,NaN,1.0,NaN,2018,NaN,NaN,No
950803659465658368,1,Our Chief People Officer David Almeida talks a...,https://twitter.com/abinbev/status/95080365946...,AB InBev,@abinbev,0,0,7,21,-,0,2018-01-09 19:57:12,https://pbs.twimg.com/media/DTHu2wQWkAEpVbv.jpg,NaN,1.0,NaN,2018,NaN,NaN,No


In [11]:
tweets_df['openai_tag'].value_counts()

openai_tag
No     347238
Yes     40531
Name: count, dtype: int64

In [12]:
# Filtrar los tweets etiquerados como ESG por ChatGPT ('Yes' en la columna 'openai_tag')
tweets_df = tweets_df[tweets_df['openai_tag'] == 'Yes']


In [13]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40531 entries, 955452656050495488 to 1130783217756323968
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Empresa        40531 non-null  int64         
 1   tweetText      40531 non-null  string        
 2   tweetURL       40531 non-null  object        
 3   tweetAuthor    40531 non-null  object        
 4   handle         40531 non-null  object        
 5   replyCount     40531 non-null  int64         
 6   quoteCount     40531 non-null  int64         
 7   retweetCount   40531 non-null  int64         
 8   likeCount      40531 non-null  int64         
 9   views          26988 non-null  object        
 10  bookmarkCount  40531 non-null  int64         
 11  createdAt      40531 non-null  datetime64[ns]
 12  allMediaURL    25410 non-null  object        
 13  videoURL       8182 non-null   object        
 14  Idioma_1       40318 non-null  float64      

In [14]:
# Veo dimensiones del dataframe
# Obtener el número de tweets (filas) y campos por tweet (columnas)
num_tweets, num_fields = tweets_df.shape

# Imprimir el resultado con formato
print(f"El DataFrame contiene {num_tweets} tweets y {num_fields} campos por tweet.")


El DataFrame contiene 40531 tweets y 20 campos por tweet.


In [15]:
#  Los tweets están en varios idiomas, veo cuántos tweets hay por idioma
tweets_df['Idioma_1'].value_counts()

Idioma_1
1.0    23861
2.0     9095
3.0     4838
5.0     1905
4.0      619
Name: count, dtype: int64

In [16]:


# Lista de palabras a buscar proporcionadas por Fenando
palabras = ['medio ambiente', 'sostenible', 'ecológico', 'verde', 'biodegradable', 'compostable']


# Listado de palabras en inglés, español, francés, alemán e italiano generadas a partir de las proporcionadas por Fernando usando ChatGPT
# Cada palabra aparece una única vez y en la fila del sinónimo correspondiente

# Lista de listas agrupadas por sinónimos para cada uno de los idiomas
words_in_english = [
    ["environment", "nature", "ecosystem"],  # Synonyms for 'environment'
    ["sustainable", "renewable", "viable"],  # Synonyms for 'sustainable'
    ["ecological", "eco-friendly"],  # Synonyms for 'ecological'
    ["green", "natural", "organic"],  # Synonyms for 'green'
    ["biodegradable", "decomposable", "environmentally-friendly"],  # Synonyms for 'biodegradable'
    ["compostable", "recyclable", "degradable"]  # Synonyms for 'compostable'
]

words_in_spanish = [
    ["medio ambiente", "naturaleza", "entorno", "ecosistema", "ambiente"],  # Sinónimos de 'medio ambiente'
    ["sostenible", "sustentable", "renovable", "viable", "ecológico"],  # Sinónimos de 'sostenible'
    ["ambiental", "ecoamigable", "natural"],  # Sinónimos de 'ecológico'
    ["verde", "orgánico", "sano"],  # Sinónimos de 'verde'
    ["biodegradable", "degradable", "ecofriendly"],  # Sinónimos de 'biodegradable'
    ["compostable", "reciclable", "desechable", "descomponible"]  # Sinónimos de 'compostable'
]

words_in_french = [
    ["environnement", "nature", "écosystème", "milieu", "entourage"],  # Synonyms for 'medio ambiente'
    ["durable", "soutenable", "viable", "renouvelable", "écologique"],  # Synonyms for 'sostenible'
    ["environnemental", "éco-responsable", "naturel"],  # Synonyms for 'ecológico'
    ["vert", "biologique", "sain"],  # Synonyms for 'verde'
    ["biodégradable", "décomposable", "respectueux de l'environnement"],  # Synonyms for 'biodegradable'
    ["compostable", "recyclable", "dégradable"]  # Synonyms for 'compostable'
]

words_in_german = [
    ["umwelt", "natur", "ökosystem", "umfeld", "umgebung"],  # Synonyms for 'medio ambiente'
    ["nachhaltig", "tragfähig", "erneuerbar", "beständig", "ökologisch"],  # Synonyms for 'sostenible'
    ["umweltfreundlich", "natürlich", "umweltbezogen"],  # Synonyms for 'ecológico'
    ["grün", "biologisch", "gesund"],  # Synonyms for 'verde'
    ["biologisch abbaubar", "abbaubar", "umweltfreundlich"],  # Synonyms for 'biodegradable'
    ["kompostierbar", "recycelbar", "zerlegbar"]  # Synonyms for 'compostable'
]


words_in_italian = [
    ["ambiente", "natura", "ecosistema", "contesto", "circondario"],  # Synonyms for 'medio ambiente'
    ["sostenibile", "rinnovabile", "duraturo", "fattibile"],  # Synonyms for 'sostenible'
    ["ecologico", "ambientale", "eco-compatibile", "naturale"],  # Synonyms for 'ecológico'
    ["verde", "organico", "salutare"],  # Synonyms for 'verde'
    ["biodegradabile", "degradabile", "amico dell'ambiente"],  # Synonyms for 'biodegradable'
    ["compostabile", "riciclabile", "disintegrabile"]  # Synonyms for 'compostable'
]

# Diccionario con el nombre de los idiomas como clave y la lista de palabras como valor
words_by_language = {
    "inglés": words_in_english,
    "español": words_in_spanish,
    "francés": words_in_french,
    "alemán": words_in_german,
    "italiano": words_in_italian
}



In [17]:
# Idiomas detectados en fichero excell. Los números se corresponden a la comlumna Idioma_1

idiomas_dict = {
    1: "inglés",
    2: "español",
    3: "francés",
    4: "alemán",
    5: "italiano"
}



# Crear un diccionario para almacenar DataFrames separados por idioma
dataframes_by_language = {}

# Dividir el DataFrame principal en sub-DataFrames por idioma
for idioma_id, idioma_name in idiomas_dict.items():
    dataframes_by_language[idioma_name] = tweets_df[tweets_df['Idioma_1'] == idioma_id].copy()


In [18]:
# Procesar cada idioma
for idioma, df in dataframes_by_language.items():
    # Obtener las listas de palabras para este idioma
    words_list = words_by_language.get(idioma, [])
    
    # Crear columnas basadas en las primeras palabras de cada lista de sinónimos
    for word_group in words_list:
        column_name = word_group[0]  # Usar la primera palabra como nombre de columna
        
        # Crear un patrón regex para todas las palabras del grupo, ignorando puntuación
        pattern = rf'\b({"|".join(re.escape(word) for word in word_group)})[^\w]*\b'
        
        # Contar las apariciones de todas las palabras del grupo en cada tweet
        df[column_name] = df['tweetText'].str.lower().str.count(pattern)
        
    
    # Reemplazar el DataFrame en el diccionario con el actualizado
    dataframes_by_language[idioma] = df

# Mostrar las claves del diccionario para verificar el procesamiento
dataframes_by_language.keys()

dict_keys(['inglés', 'español', 'francés', 'alemán', 'italiano'])

In [19]:
dataframes_by_language['inglés'].columns

Index(['Empresa', 'tweetText', 'tweetURL', 'tweetAuthor', 'handle',
       'replyCount', 'quoteCount', 'retweetCount', 'likeCount', 'views',
       'bookmarkCount', 'createdAt', 'allMediaURL', 'videoURL', 'Idioma_1',
       'Idioma_2', 'Año', 'Unnamed: 20', 'Unnamed: 21', 'openai_tag',
       'environment', 'sustainable', 'ecological', 'green', 'biodegradable',
       'compostable'],
      dtype='object')

In [21]:
columnas_ingles = ['environment', 'sustainable', 'ecological', 'green', 'biodegradable', 'compostable']
dataframes_by_language['inglés'].loc[:, columnas_ingles].sum()

environment      1045
sustainable      4421
ecological        141
green            1469
biodegradable      42
compostable       153
dtype: Int64

In [28]:
100 * (dataframes_by_language['inglés'].loc[:, columnas_ingles] != 0).sum()/dataframes_by_language['inglés'].shape[0]

environment       4.161603
sustainable      17.199614
ecological        0.582541
green             5.636813
biodegradable     0.171829
compostable       0.507104
dtype: Float64

In [22]:
dataframes_by_language['español'].columns

Index(['Empresa', 'tweetText', 'tweetURL', 'tweetAuthor', 'handle',
       'replyCount', 'quoteCount', 'retweetCount', 'likeCount', 'views',
       'bookmarkCount', 'createdAt', 'allMediaURL', 'videoURL', 'Idioma_1',
       'Idioma_2', 'Año', 'Unnamed: 20', 'Unnamed: 21', 'openai_tag',
       'medio ambiente', 'sostenible', 'ambiental', 'verde', 'biodegradable',
       'compostable'],
      dtype='object')

In [24]:
columnas_español = ['medio ambiente', 'sostenible', 'ambiental', 'verde', 'biodegradable', 'compostable']
dataframes_by_language['español'].loc[:, columnas_español].sum()

medio ambiente     516
sostenible        1908
ambiental          130
verde              807
biodegradable        7
compostable          8
dtype: Int64

In [29]:
100 * (dataframes_by_language['español'].loc[:, columnas_español] != 0).sum() / dataframes_by_language['español'].shape[0]

medio ambiente    5.409566
sostenible        19.95602
ambiental         1.418362
verde             8.620121
biodegradable     0.076965
compostable        0.08796
dtype: Float64